In [18]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder


In [19]:
df=pd.read_csv("Bank Customer Churn Prediction.csv")

In [20]:
df.head(5)

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
#drop irrilrval columns
df.drop('customer_id',axis=1,inplace=True)
#df.drop('country',axis=1,inplace=True)
#df.drop('gender',axis=1,inplace=True)


In [22]:
df.head(5)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
##Encode categorical variables 
label_encoder_gender=LabelEncoder()
df['gender']=label_encoder_gender.fit_transform(df['gender'])

In [24]:
df.head(5)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [25]:
#onehot encoding
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_country=OneHotEncoder()
counrty_enc=onehot_encoder_country.fit_transform(df[['country']])

In [26]:
counrty_enc.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [27]:
onehot_encoder_country.get_feature_names_out(['country'])

array(['country_France', 'country_Germany', 'country_Spain'], dtype=object)

In [29]:
geo_encoder_df=pd.DataFrame(counrty_enc.toarray(),columns=onehot_encoder_country.get_feature_names_out(['country']))

In [30]:
geo_encoder_df

,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [31]:
## Combine one hot encode data with original data ?
data=pd.concat([df.drop(['country'],axis=1),geo_encoder_df],axis=1)

In [32]:
data.head(5)

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_France,country_Germany,country_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [1]:
## Save the encoer and scaler

with open("label_enc_file.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open("onehot_enc_file.pkl",'wb') as file:
    pickle.dump(onehot_encoder_country,file)

NameError: name 'pickle' is not defined